In [ ]:
import pandas as pd

# Load datasets (replace 'path_to_financial_ledger.csv' and 'path_to_useeio.csv' with actual paths)
financial_ledger_df = pd.read_csv('path_to_financial_ledger.csv')
useeio_df = pd.read_csv('path_to_useeio.csv')

# Assume financial ledger already has a column 'mapped_naics_code' mapped to NAICS codes
# and 'amount' which is the monetary value of transactions

# Ensure the 'mapped_naics_code' column is of the same type in both datasets
financial_ledger_df['mapped_naics_code'] = financial_ledger_df['mapped_naics_code'].astype(str)
useeio_df['naics_code'] = useeio_df['naics_code'].astype(str)

# Display initial information about the datasets
print("Financial Ledger Dataset Info:")
print(financial_ledger_df.info())
print("\nUSEEIO Dataset Info:")
print(useeio_df.info())

# Emission Estimation
# Merge the financial ledger with USEEIO dataset based on 'mapped_naics_code' to get the emission factors
merged_df = pd.merge(financial_ledger_df, useeio_df, left_on='mapped_naics_code', right_on='naics_code', how='inner')

# Calculate emissions for each transaction
# Assuming the USEEIO dataset has a column 'emission_factor' which is emissions per monetary unit
merged_df['emissions'] = merged_df['amount'] * merged_df['emission_factor']

# Aggregation
# Aggregate emissions by 'mapped_naics_code'
aggregated_emissions = merged_df.groupby('mapped_naics_code')['emissions'].sum().reset_index()

# Reporting
# Generate a summary report
summary_report = aggregated_emissions.copy()
summary_report.columns = ['NAICS Code', 'Total Emissions']

# Display the summary report
print("\nSummary Report:")
print(summary_report)

# Output
# Save the summary report to a CSV file
summary_report.to_csv('scope3_emissions_summary.csv', index=False)

# For visualization, you can use matplotlib or seaborn
import matplotlib.pyplot as plt
import seaborn as sns

# Plot total emissions by NAICS code
plt.figure(figsize=(12, 8))
sns.barplot(data=summary_report, x='NAICS Code', y='Total Emissions', palette='viridis')
plt.xticks(rotation=90)
plt.title('Total Scope 3 Emissions by NAICS Code')
plt.xlabel('NAICS Code')
plt.ylabel('Total Emissions')
plt.tight_layout()
plt.savefig('scope3_emissions_by_naics_code.png')
plt.show()
